In [ ]:
import os

from collections import defaultdict
from itertools import product

import pandas as pd
import requests

In [ ]:
base = os.environ['BUG_FREE_EUREKA_BASE']
roster_path = os.path.join(base, 'grading', 'roster.tsv')

In [ ]:
assignment_1 = ['DNA', 'RNA', 'GC', 'HAMM', 'REVC']
assignment_2 = ['INI', 'DBPR', 'FIB', 'IPRB']
assignment_3 = ['FRMT', 'PROT', 'SUBS', 'CONS', 'FIBD', 'GRPH', 'IEV', 'MPRT', 'MRNA', 'PERM', 'PRTM']
assignment_all = assignment_1 + assignment_2 + assignment_3

In [ ]:
roster = pd.read_csv(roster_path, index_col=0, sep='\t')
roster.head()

In [ ]:
done = defaultdict(dict)

for name, assignment in product(roster.index, assignment_all):
    done[name][assignment] = 0

In [ ]:
for name, github, rosalind in roster.itertuples():
    print('Checking {}'.format(name))

    res = requests.get(rosalind)

    for line in res.iter_lines():
        line = line.decode('utf-8').strip()
        if 'badge-success' not in line:
            continue
        success = line[1 + line.find('>'): line.find('<', 2)]
        done[name][success] = 1

In [ ]:
results = pd.DataFrame(done).T
results[results.isnull()] = 0 # sanitize from missing assignments
results = results.astype(int)

In [ ]:
print('Assignment 1 has {} tasks'.format(len(assignment_1)))
results[assignment_1].sum(axis=1)

In [ ]:
print('Assignment 2 has {} tasks'.format(len(assignment_2)))
results[assignment_2].sum(axis=1)

In [ ]:
print('Assignment 3 has {} tasks'.format(len(assignment_3)))
results[assignment_3].sum(axis=1)